In [1]:
import gesture_framework
import json
import tensorflow as tf
import numpy as np
import copy

**Create Framework Object**

In [2]:
gf = gesture_framework.gesture_framework()

**Read Datasets**

In [3]:
with open('gestures/dataset.json', 'r') as f:
    originals = np.asarray(json.load(f))
with open('gestures/dataset_np.json', 'r') as f:
    originals_np = json.load(f)
    for i in range(len(originals_np)):
        o = (np.asarray(originals_np[i])+1)/2
        originals_np[i] = [o.tolist()]
with open('gestures/gestures_good.json', 'r') as f:
    synth_data1 = np.asarray(json.load(f))
with open('gestures/gestures_good_np.json', 'r') as f:
    synth_data1_np = json.load(f)
    for i in range(len(synth_data1_np)):
        synth_data1_np[i] = [synth_data1_np[i]]
with open('gestures/gestures_mid.json', 'r') as f:
    synth_data2 = np.asarray(json.load(f))
with open('gestures/gestures_mid_np.json', 'r') as f:
    synth_data2_np = json.load(f)
    for i in range(len(synth_data2_np)):
        synth_data2_np[i] = [synth_data2_np[i]]
with open('gestures/gestures_bad.json', 'r') as f:
    synth_data3 = np.asarray(json.load(f))
with open('gestures/gestures_bad_np.json', 'r') as f:
    synth_data3_np = json.load(f)
    for i in range(len(synth_data3_np)):
        synth_data3_np[i] = [synth_data3_np[i]]
with open('gestures/labels.json', 'r') as f:
    labels = np.asarray(json.load(f))

**Get Classifiers**

In [4]:
classifier_tstr_empty = classifier_dau_empty = 'classifier/classifier_empty.h5'
classifier_tstr_trained = tf.keras.models.load_model('classifier/device_classifier_7-19_new_new.h5')
classifier_trts = tf.keras.models.load_model('classifier/device_classifier_7-19_new_new.h5')

classifier_td_long = tf.keras.models.load_model('classifier/td_long.h5', compile=False)
classifier_td_long.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001), loss='mean_squared_error', metrics=[tf.keras.metrics.MeanSquaredError()])
shape_long = [3,3]
classifier_td_mid = tf.keras.models.load_model('classifier/td_mid.h5', compile=False)
classifier_td_mid.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001), loss='mean_squared_error', metrics=[tf.keras.metrics.MeanSquaredError()])
shape_mid = [3,2]
classifier_td_short = tf.keras.models.load_model('classifier/td_small.h5', compile=False)
classifier_td_short.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001), loss='mean_squared_error', metrics=[tf.keras.metrics.MeanSquaredError()])
shape_short = [2,2]

c:\Users\marvi\anaconda3\envs\no_tfGAN_GANs\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


**Representativeness**

In [5]:
bin_number = 50
kl_d1 = gf.compare_physical_properties(originals_np, synth_data1_np, bin_number)
kl_d2 = gf.compare_physical_properties(originals_np, synth_data2_np, bin_number)
kl_d3 = gf.compare_physical_properties(originals_np, synth_data3_np, bin_number)

In [ ]:
print(kl_d1)

In [ ]:
print(kl_d2)

In [ ]:
print(kl_d3)

In [ ]:
wd1 = gf.get_wasserstein_distance(originals_np[:250], synth_data1_np[:250])
wd2 = gf.get_wasserstein_distance(originals_np[:250], synth_data2_np[:250])
wd3 = gf.get_wasserstein_distance(originals_np[:250], synth_data3_np[:250])
print(wd1, wd2, wd3)

In [ ]:
epochs = 15
c = copy.copy(classifier_tstr_empty)
tstr1 = gf.tstr(originals, synth_data1, labels, c, classifier_tstr_trained, epochs)
tstr2 = gf.tstr(originals, synth_data1, labels, c, classifier_tstr_trained, epochs)
tstr3 = gf.tstr(originals, synth_data1, labels, c, classifier_tstr_trained, epochs)
print(tstr1, tstr2, tstr3)

**Novelty**

In [ ]:
novelty1 = gf.nnad_novelty(synth_data1_np[:250], originals_np[:250])
novelty2 = gf.nnad_novelty(synth_data2_np[:250], originals_np[:250])
novelty3 = gf.nnad_novelty(synth_data3_np[:250], originals_np[:250])
print(novelty1, novelty2, novelty3)

**Realism**

In [ ]:
trts1 = gf.trts(synth_data1, labels, classifier_trts)
trts2 = gf.trts(synth_data2, labels, classifier_trts)
trts3 = gf.trts(synth_data3, labels, classifier_trts)
print(trts1, trts2, trts3)

**Diversity**

In [ ]:
diversity1 = gf.nnad_diversity(synth_data1_np[:250])
diversity2 = gf.nnad_diversity(synth_data2_np[:250])
diversity3 = gf.nnad_diversity(synth_data3_np[:250])
print(diversity1, diversity2, diversity3)

In [ ]:
portions = [0, 0.1, 0.3, 0.5]
epochs = 15
dau1 = gf.dau(originals, synth_data1, labels, classifier_dau_empty, portions, epochs)
dau2 = gf.dau(originals, synth_data2, labels, classifier_dau_empty, portions, epochs)
dau3 = gf.dau(originals, synth_data3, labels, classifier_dau_empty, portions, epochs)

In [ ]:
print(dau1)

In [ ]:
print(dau2)

In [ ]:
print(dau3)

**Coherence**

In [ ]:
classifiers = [classifier_td_long, classifier_td_mid, classifier_td_short]
shapes = [shape_long, shape_mid, shape_short]
s1 = []
for i in range(len(synth_data1_np)):
    s1.append(synth_data1_np[i][0])
td1 = gf.temporal_depencies(s1, classifiers, shapes, 7)
s2 = []
for i in range(len(synth_data2_np)):
    s2.append(synth_data2_np[i][0])
td2 = gf.temporal_depencies(s2, classifiers, shapes, 7)
s3 = []
for i in range(len(synth_data3_np)):
    s3.append(synth_data3_np[i][0])
td3 = gf.temporal_depencies(s3, classifiers, shapes, 7)
print(td1, td2, td3)